# Imports

In [ ]:
from prepare_data import sayhello

ss = sayhello()
print(ss)

In [1]:
import sqlite3
import pandas as pd
import networkx as nx
import matplotlib
import matplotlib.pyplot as plt

SQL_LITE_DB_PATH = "D:\\Repos\\ut-health\\mimic-iii-gz\\mimic3.db"

# Connect to the SQLite database in the file at the given path d:\data\mimic.db
conn = sqlite3.connect(SQL_LITE_DB_PATH)

# print version of sqlite
cursor = conn.cursor()
cursor.execute("SELECT sqlite_version()")
print("SQLite version: %s" % cursor.fetchone())

# close connection
conn.close()

# print pandas version
print("Pandas version: %s" % pd.__version__)

# print matplotlib version
print(f"Matplotlib version: {matplotlib.__version__}")

print(f"NetworkX version: {nx.__version__}")

# Set pandas display options to show all columns and rows without truncation
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
print("\nRemoved truncation of columns")


SQLite version: 3.49.1
Pandas version: 2.2.3
Matplotlib version: 3.10.1
NetworkX version: 3.4.2

Removed truncation of columns


# Load Data

In [2]:
# load patient data
conn = sqlite3.connect(SQL_LITE_DB_PATH)

query = "SELECT * FROM patients"
patients_df = pd.read_sql_query(query, conn)

query = "SELECT * FROM d_icd_diagnoses"
d_icd_diagnoses_df = pd.read_sql_query(query, conn)

query = "SELECT * FROM diagnoses_icd"
diagnoses_icd_df = pd.read_sql_query(query, conn)

query = "SELECT * FROM admissions"
admissions_df = pd.read_sql_query(query, conn)

conn.close()

# Subselect size of data
Reduces the number of patients, picks the latest admission (hadm_id) for the patient and picks only those diagnoses belonging to the hospital admission.

In [3]:
# print the number of patients and diagnoses
print(f"Number of patients: {len(patients_df)}")
print(f"Number of diagnoses: {len(diagnoses_icd_df)}")
print(f"Number of admissions: {len(admissions_df)}")

pt_frac = 0.0001  # 0.01% of the patients gives 5 patients, 52 diagnoses
pt_frac = 0.0002  # 0.02% of the patients gives 9 patients, 110 diagnoses
pt_frac = 0.0003  # 0.02% of the patients gives 14 patients, 188 diagnoses
pt_frac = 0.0015  # 0.02% of the patients gives 14 patients, 188 diagnoses

# select 0.01% of the patients randomly
patients_df = patients_df.sample(frac=pt_frac, random_state=1)

# add DOB to admissions_df
admissions_df = admissions_df.merge(patients_df[['SUBJECT_ID', 'DOB']], left_on='SUBJECT_ID', right_on='SUBJECT_ID', how='inner', suffixes=('_adm', '_pt'))

# sort admissions_df by SUBJECT_ID and ADMITTIME in ascending order
admissions_df['ADMITTIME'] = pd.to_datetime(admissions_df['ADMITTIME'])
admissions_df = admissions_df.sort_values(by=['SUBJECT_ID', 'ADMITTIME'])

# keep the newest admission (HADM_ID) for each patient, drop the rest of the admissions
admissions_df = admissions_df.groupby('SUBJECT_ID').last().reset_index()

# keep only diagnoses that belong to HADM_IDs in admissions_df
diagnoses_icd_df = diagnoses_icd_df[diagnoses_icd_df['HADM_ID'].isin(admissions_df['HADM_ID'])]

# print the number of patients and diagnoses
print(f"Updated number of patients: {len(patients_df)}")
print(f"Updated number of diagnoses: {len(diagnoses_icd_df)}")
print(f"Updated number of admissions: {len(admissions_df)}")

# TODO: fix the HADM_ID issue, only keep diagnosies that are from the latest admissions under subject id, and drop the rest, do same for admissions_df

Number of patients: 46520
Number of diagnoses: 651047
Number of admissions: 58976
Updated number of patients: 70
Updated number of diagnoses: 813
Updated number of admissions: 70


# Feature Engineering
Adds age bucket feature and hadm_id to patient dataframe

In [4]:
# log data
print(f"Number of patients: {len(patients_df)}")

# convert to integer from datetime type to calculate age (without overflow)
admissions_df['DOB'] = admissions_df['DOB'].astype('datetime64[s]').astype(int)
admissions_df['ADMITTIME'] = admissions_df['ADMITTIME'].astype('datetime64[s]').astype(int)

# calculate age at admission
admissions_df['AGE'] = (admissions_df['ADMITTIME'] - admissions_df['DOB']) / 86400 / 365

# create age intervals in 5 year increments, of type int
admissions_df['AGE_BUCKET'] = (admissions_df['AGE'] // 5).astype(int) * 5

# set age bucket to 95 where age is greater than 90
admissions_df.loc[admissions_df['AGE_BUCKET'] > 90, 'AGE_BUCKET'] = 95

# add AGE_BUCKET to patients_df
patients_df = patients_df.merge(admissions_df[['SUBJECT_ID', 'HADM_ID', 'AGE_BUCKET']], left_on='SUBJECT_ID', right_on='SUBJECT_ID', how='inner', suffixes=('_pt', '_adm'))

print(f"Updated number of patients (with age bucket): {len(patients_df)}")

Number of patients: 70
Updated number of patients (with age bucket): 70


In [5]:
patients_df.head(5)

,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG,HADM_ID,AGE_BUCKET
0,3850,4074,M,2121-08-01 00:00:00,2204-02-06 00:00:00,2204-02-06 00:00:00,2204-02-06 00:00:00,1,137421,80
1,44143,90889,M,2095-04-29 00:00:00,None,None,None,0,168745,55
2,39453,72753,M,2042-06-01 00:00:00,None,None,None,0,144029,85
3,37377,64908,M,2108-04-22 00:00:00,None,None,None,0,172851,75
4,38824,70273,F,2090-05-28 00:00:00,None,None,None,0,102912,75


# Create Graph

## Create patient nodes

In [ ]:
# Create a graph
G = nx.Graph()

i = 0

for _, row in patients_df.iterrows():

    # Add a node with the patient ID as the node identifier and gender as an attribute, also add dob as an attribute
    G.add_node(row['SUBJECT_ID'], gender=row['GENDER'], age=row['DOB'], age_bucket=row['AGE_BUCKET'], hadm_id=row['HADM_ID'])

    i+=1
    if i % 2000 == 0:
        print(f"Added {i} patient nodes, percent: {i / len(patients_df) * 100:.2f}%")

# number of nodes in the graph
print(f"Number of nodes in the graph: {G.number_of_nodes()}")

## Create diagnosis to patient mappings

In [ ]:
# group diagnoses by icd9 codes, and create a dictionary with each icd9 code as the key and the subject_ids as the values
diagnoses_to_patients = diagnoses_icd_df.groupby('ICD9_CODE')['SUBJECT_ID'].apply(set).to_dict()

# print 1 random key and value from the dictionary
print(f"Sample keys and values from the dictionary:")
print(f"\t\t{list(diagnoses_to_patients.items())[12]}")
print(f"\t\t{list(diagnoses_to_patients.items())[15]}")

# print size of the dictionary and counts of values
print(f"Count of the number of keys (icd9s) in the dictionary: {len(diagnoses_to_patients)}")
print(f"Count of the number of values (subject_ids) in all keys: {sum(len(v) for v in diagnoses_to_patients.values())}")

### Validation code

In [ ]:
# Validation
unique_subject_ids = len(set([item for sublist in diagnoses_to_patients.values() for item in sublist]))

# count number of unique subject_ids across all values in the dictionary
print(f"Count of the number of unique subject_ids across all keys: {unique_subject_ids}")

# print number of patients with diagnosis 0389
print(f"Number of patients with diagnosis 0389: {len(diagnoses_to_patients.get('0389', []))}")

# assert that the number of unique subject_ids is equal to the number of patients in the patients_df dataframe
assert unique_subject_ids == len(patients_df), f"Number of unique subject_ids in the dictionary ({unique_subject_ids}) is not equal to the number of patients in the patients_df dataframe ({len(patients_df)})"


## Add diagnoses edges between patients

In [ ]:
# print the number of edges in the graph
print(f"Number of edges: {len(G.edges())}")

# print the number of nodes in the graph
print(f"Number of nodes: {len(G.nodes())}")

In [ ]:
i=0
ONCE = True

# add diagnosis/icd9 edge between patients
for icd9_code, patients in diagnoses_to_patients.items():

    # get SHORT_TITLE from d_icd_diagnoses_df for the icd9 code
    short_title = d_icd_diagnoses_df[d_icd_diagnoses_df['ICD9_CODE'] == icd9_code]['SHORT_TITLE'].values

    if len(short_title) == 0:
        short_title = "Unknown"
    else:
        # convert to a string
        short_title = short_title[0]

    if ONCE:
        print(f"ICD9 code: {icd9_code}, short title: {short_title}")
        ONCE = False

    if len(patients) < 2:
        continue

    # if there are more than 1 patients with the same diagnosis, add edges between them
    patients = list(patients)

    combos = [(patients[i], patients[j]) for i in range(len(patients)) for j in range(i + 1, len(patients))]
    G.add_edges_from(combos, icd9=icd9_code, short_title=short_title)
    i += 1
    if i % 4000 == 0:
        print(f"Added {i} edges, percent: {i / len(diagnoses_to_patients) * 100:.2f}%")


In [ ]:
# print the number of edges in the graph
print(f"Number of edges: {len(G.edges())}")

# print the number of nodes in the graph
print(f"Number of nodes: {len(G.nodes())}")

# Constants

In [ ]:
import os

CURR_DIR_PATH = os.getcwd()

PICKLE_FILE_NAME = None
HTML_FILE_NAME = None

if G.number_of_nodes() < 10:
    PICKLE_FILE_NAME = f"{CURR_DIR_PATH}\\pickle\\patients_graph_max_10_nodes.gpickle"
    HTML_FILE_NAME = f"{CURR_DIR_PATH}\\pyvis\\graph_max_10_nodes.html"
elif G.number_of_nodes() < 50:
    PICKLE_FILE_NAME = f"{CURR_DIR_PATH}\\pickle\\patients_graph_max_50_nodes.gpickle"
    HTML_FILE_NAME = f"{CURR_DIR_PATH}\\pyvis\\graph_max_50_nodes.html"
elif G.number_of_nodes() < 100:
    PICKLE_FILE_NAME = f"{CURR_DIR_PATH}\\pickle\\patients_graph_max_100_nodes.gpickle"
    HTML_FILE_NAME = f"{CURR_DIR_PATH}\\pyvis\\graph_max_100_nodes.html"
elif G.number_of_nodes() < 1000:
    PICKLE_FILE_NAME = f"{CURR_DIR_PATH}\\pickle\\patients_graph_max_1000_nodes.gpickle"
    HTML_FILE_NAME = f"{CURR_DIR_PATH}\\pyvis\\graph_max_1000_nodes.html"
elif G.number_of_nodes() < 10000:
    PICKLE_FILE_NAME = f"{CURR_DIR_PATH}\\pickle\\patients_graph_max_10000_nodes.gpickle"
    HTML_FILE_NAME = f"{CURR_DIR_PATH}\\pyvis\\graph_max_10000_nodes.html"
else:
    PICKLE_FILE_NAME = f"{CURR_DIR_PATH}\\pickle\\patients_graph_max_46000_nodes.gpickle"
    HTML_FILE_NAME = f"{CURR_DIR_PATH}\\pyvis\\graph_max_46000_nodes.html"

print(f"Pickle file name: {PICKLE_FILE_NAME}")
print(f"HTML file name: {HTML_FILE_NAME}")

# Pyvis Graph

In [ ]:
# use pyvis to draw the graph
from pyvis.network import Network


nt = Network(height="1200px", width="100%", notebook=False, directed=False, cdn_resources="remote")

print("created pyvis network")
nt.from_nx(G)

# for node in nt.nodes:
#     print(f"Node ID: {node['id']}")

# add node attributes to the pyvis network
for node in nt.nodes:
    node['title'] = f"Patient ID: {node['id']}\nGender: {node['gender']}\nAge Bucket: {node['age_bucket']}"
    node['label'] = f"{node['id']}"
    node['font'] = {'size': 5}

# add edge attributes to the pyvis network
for edge in nt.edges:
    edge['title'] = f"ICD9 Code: {edge['icd9']}"
    edge['label'] = f"{edge['short_title']}"
    # set font size to 5
    edge['font'] = {'size': 5}

print("added nx graph to pyvis network")
print(f"file name: {HTML_FILE_NAME}")
nt.toggle_physics(False)
nt.show_buttons()
nt.generate_html(HTML_FILE_NAME)
nt.save_graph(HTML_FILE_NAME)
# nt.show(HTML_FILE_NAME)
print("saved pyvis graph to file")


# Save Graph to Disk

## pickle

In [ ]:
import pickle

# Save the graph to a pickle file
with open(f"{PICKLE_FILE_NAME}", "wb") as f:
    pickle.dump(G, f)
    print("Graph saved to pickle file")


# Create Similarity Graph Dataframe

## Load graph from pickle file

In [ ]:
# Load the graph from a pickle file
with open(f"{PICKLE_FILE_NAME}", "rb") as f:
    G = pickle.load(f)
    print("Graph loaded from pickle file")

print(f"Number of nodes: {G.number_of_nodes()}")
print(f"Number of edges: {G.number_of_edges()}")

## Create similar patient graph
This graph contains patient nodes, and only those patients are connected with edges that are similar to each other

In [ ]:
# NOT NEEDED

from scipy.sparse import coo_matrix

patient_to_icd9 = diagnoses_icd_df.groupby('SUBJECT_ID')['ICD9_CODE'].apply(set).to_dict()
# get unique list of icd9 codes for all patients
all_icd9_codes = set()
for codes in patient_to_icd9.values():
    all_icd9_codes.update(codes)

icd9_codes = sorted(all_icd9_codes)

# map subject_id to a row index
patients = sorted(patient_to_icd9.keys())
patient_to_index = {pid: i for i, pid in enumerate(patients)}

# map icd9_code to a column index
code_to_index = {code: i for i, code in enumerate(icd9_codes)}

row_indices = []
col_indices = []
data = []

# fill row_indices, col_indices, and data
for pid, codes in patient_to_icd9.items():
    for code in codes:
        row_indices.append(patient_to_index[pid])
        col_indices.append(code_to_index[code])
        data.append(1)

# create the sparse matrix
icd9_sparse_matrix = coo_matrix(
    (data, (row_indices, col_indices)), 
    shape=(len(patients), len(icd9_codes))
)

# create a CSR matrix from the COO matrix
icd9_csr_matrix = icd9_sparse_matrix.tocsr()

# print the shape of the matrix
print(f"Shape of the matrix: {icd9_csr_matrix.shape}")


## Create pivot that maps patients to all unique icd9 codes

In [6]:
print(f"Number of patients: {len(patients_df)}")

patient_to_icd9 = diagnoses_icd_df.groupby('SUBJECT_ID')['ICD9_CODE'].apply(set).to_dict()

# get unique list of icd9 codes for all patients
all_icd9_codes = set()
for codes in patient_to_icd9.values():
    all_icd9_codes.update(codes)

# create dataframe with icd9 codes as columns and patients as rows
icd9_df = pd.DataFrame(columns=list(all_icd9_codes), index=patients_df['SUBJECT_ID'])

# fill the dataframe with 1s and 0s
for patient_id, codes in patient_to_icd9.items():
    icd9_df.loc[patient_id] = [1 if code in codes else 0 for code in icd9_df.columns]

icd9_df.head(5)


Number of patients: 70


,41189,36846,5180,77211,V440,29420,V4986,81305,40390,5755,3529,7746,78930,E9394,V4581,V1271,7464,7757,2515,4139,80708,51851,6929,99662,319,2409,2867,4240,4111,56400,1572,5932,43889,42732,7109,76527,4870,E911,5752,5758,27800,2554,V4502,77181,5990,E8790,43491,80501,E9361,5762,30000,57510,78899,769,V850,41401,2930,0971,51881,28419,78009,4592,7809,5178,7856,4241,5680,43310,V441,78321,514,V290,78060,V8801,5070,45341,V0981,76528,7812,E956,74783,3572,27801,6258,7470,V4501,V090,56722,4479,30391,56210,9162,2441,04149,3688,5728,V5861,1578,51882,78702,56089,7211,9975,412,55092,V422,591,42613,23871,78052,5609,29570,5856,87349,1532,7455,9340,2639,5761,3659,4264,86813,2764,99831,99592,E9320,5789,42833,0389,5191,5570,1550,99883,E8788,99594,0092,27650,2749,42821,042,78820,9962,0088,5854,53290,1962,V667,75839,42823,76514,E8786,2859,99672,56981,V4364,41072,72691,E8810,V1003,41011,76513,7786,V5865,7991,78099,2765,25002,V1042,V4511,3484,V721,3970,2948,5799,42822,5997,2875,60000,V1005,76518,9982,79029,2449,7707,4019,78720,5768,7766,E8798,486,52100,515,70703,1911,2720,7140,E8495,41400,41041,V462,7249,3079,78951,2760,28529,4280,29410,E8889,2752,59970,45182,V3101,7802,53560,5601,0311,8052,V443,29690,501,51633,77081,8601,43820,5304,2532,19889,76523,7906,99591,4271,27730,8604,3051,1985,3439,1369,58381,5853,1536,40391,5718,28860,E8120,27651,75251,V851,V5867,42731,75501,5119,99659,28521,85221,25080,56081,25012,55320,5781,32723,V3000,E9352,7104,8749,34982,56031,51884,5845,70714,99812,56983,0414,1976,5934,88122,7775,V8543,3004,7885,42832,1974,53550,7793,9992,5720,25201,3314,4739,V5866,452,5723,V1251,0413,V444,4329,99702,6823,07032,25050,2724,4267,76503,V4960,1970,E8497,3576,6930,56200,82020,79902,V420,75315,99932,5181,3899,41071,28800,99811,2841,77981,2753,72888,1983,52340,7510,1977,4959,V8741,74781,5715,V1082,3310,45621,7907,V433,53081,1978,43411,7742,30550,3485,V1087,4110,4148,29181,3061,8730,20020,29281,78552,36201,9351,33829,V1006,78551,7661,73300,E8809,4275,25060,25040,E8782,V1011,78791,V502,7716,4168,45829,78701,V1046,2630,2761,5849,25000,99800,04186,49121,V293,70702,2352,9973,9038,99602,311,E8781,72400,V1204,V053,49390,99739,53010,7473,4263,E9331,7560,03811,75322,2768,86415,V452,2536,5533,2851,30540,V1582,E915,75881,6826,99859,34831,04185,V173,34830,70705,3313,34680,60001,431,V1209,185,3558,5712,04104,4589,7706,5738,42789,2762,80375,1963,1715,79001,5933,70400,E8799,40301,80502,5859,78039,7994,5185,33182,5750,V153,E8783,9971,29620,4142,V3001,75614,V4582,2767,496
SUBJECT_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4074,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
90889,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,

## Convert patient-icd9 dataframe to a sparse matrix

In [8]:
import pandas as pd

# Convert to sparse DataFrame
sparse_df = icd9_df.astype(pd.SparseDtype("int", 0))
coo = sparse_df.sparse.to_coo()
csr = coo.tocsr()

In [10]:
# print csr matrix and compare with dataframe above to validate
print(csr[:5, :15].toarray())

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]]


## Calculate similarity between patients

In [11]:
# find the highest similarity between patients
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# calculate cosine similarity between patients
similarity_matrix = cosine_similarity(csr)
print(f"Shape of the similarity matrix: {similarity_matrix.shape}")
print(f"Similarity matrix:\n{similarity_matrix[:5, :5]}")

Shape of the similarity matrix: (70, 70)
Similarity matrix:
[[1.         0.09128709 0.25       0.15430335 0.15430335]
 [0.09128709 1.         0.         0.         0.        ]
 [0.25       0.         1.         0.07715167 0.07715167]
 [0.15430335 0.         0.07715167 1.         0.14285714]
 [0.15430335 0.         0.07715167 0.14285714 1.        ]]


## Select the top 3 matches for every patient

In [12]:
# Get the indices of the top 5 most similar patients for each patient
top_n = 3
top_n_indices = np.argsort(similarity_matrix, axis=1)[:, -top_n-1:-1][:, ::-1]

# Get the similarity scores of the top 5 most similar patients for each patient
top_n_scores = np.sort(similarity_matrix, axis=1)[:, -top_n-1:-1][:, ::-1]

# Get the patient IDs for the top 5 most similar patients for each patient
top_n_patient_ids = np.array([patients_df['SUBJECT_ID'].iloc[i] for i in top_n_indices])

# Create a DataFrame to store the results
top_n_df = pd.DataFrame(top_n_patient_ids, columns=[f"Top {i+1} Patient ID" for i in range(top_n)])

# Add the similarity scores to the DataFrame
top_n_df['Similarity Score'] = top_n_scores[:, 0]

# Add the patient IDs of the original patients to the DataFrame
top_n_df['Patient ID'] = patients_df['SUBJECT_ID'].values

# Add the index of the original patients to the DataFrame
top_n_df['Patient Index'] = np.arange(len(patients_df))

# reorder the columns in place
top_n_df = top_n_df[['Patient Index', 'Patient ID', 'Similarity Score'] + [f"Top {i+1} Patient ID" for i in range(top_n)]]

# Print the DataFrame
top_n_df.head(5)

,Patient Index,Patient ID,Similarity Score,Top 1 Patient ID,Top 2 Patient ID,Top 3 Patient ID
0,0,4074,0.350070,21280,23511,72753
1,1,90889,0.335410,50892,26708,11514
2,2,72753,0.387298,23511,11514,21280
3,3,64908,0.267261,1817,21471,23511
4,4,70273,0.253546,17906,88386,23511


# Create Similarity Pyvis Graph